# Analyze the parametric study

In [ ]:
import os


base_dir = "../../scripts/segmentation/models/carla_parametric"
config_dir = os.path.join(base_dir, "configs")
model_dir = os.path.join(base_dir, "models")

In [ ]:
import torch
from avstack.config import Config
from fov.segmentation.utils import get_dataset, get_unet_model


# load up the models and datasets
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_cfgs = []
models = {}
datasets = {}
for fname in sorted(os.listdir(model_dir))[:-1]:  # the minus 1 is for while we're training
    folder_path = os.path.join(model_dir, fname)
    if os.path.isdir(folder_path):
        cfg = Config.fromfile(os.path.join(config_dir, fname + ".py"))

        # model
        model_cfgs.append(cfg)
        models[fname] = get_unet_model(
            cfg=cfg,
            device=device,
            weight_dir=os.path.join(model_dir, fname)
        )

        # dataset
        datasets[fname] = get_dataset(
            cfg=cfg,
            device=device,
            split="test",
        )

In [ ]:
import pickle
from eval_utils import test_model_on_dataset_v2


# preset the dataset dictionary
all_results = []
all_results_res = []
n_frames_max = 200

# get the 512x512 dataset
dataset_512 = datasets["width_16_depth_3_resolution_512"][1]

# loop over models
for model_name in models:
    # pull out data structures
    model = models[model_name]
    SM, seg_dataset = datasets[model_name]

    # set names
    name_model_dataset = "CARLA-" + model_name
    name_test_dataset = "CARLA"

    # run the evaluation
    results, results_res = test_model_on_dataset_v2(
        model=model,
        name_model_dataset=name_model_dataset,
        seg_dataset=seg_dataset,
        name_test_dataset=name_test_dataset,
        n_frames_max=n_frames_max,
        evaluate_at_resolution=True,
        dataset_at_resolution=dataset_512,
    )

    # add model architecture parameters to results table
    width, depth, resolution = [int(p) for p in model_name.split("_")[1::2]]
    for result in results:
        result.update({
            "width": width,
            "depth": depth,
            "resolution": resolution,
            "model_name": model_name
        })
    for result in results_res:
        result.update({
            "width": width,
            "depth": depth,
            "resolution": resolution,
            "model_name": model_name
        })

    # store results
    all_results.extend(results)
    all_results_res.extend(results_res)

# save results
with open("parametric_train_test.p", "wb") as f:
    pickle.dump(all_results, f)
with open("parametric_train_test_eval_at_res.p", "wb") as f:
    pickle.dump(all_results_res, f)

In [ ]:
import pickle
import pandas as pd


# load in the last saved results
with open("parametric_train_test.p", "rb") as f:
    all_results = pickle.load(f)
with open("parametric_train_test_eval_at_res.p", "rb") as f:
    all_results_res = pickle.load(f)
    
# expand results where entries are lists to each frame
df_results = pd.DataFrame(all_results)
df_results_res = pd.DataFrame(all_results)

# figure directory for saving
fig_dir_metrics = "figures/parametric/metrics"
os.makedirs(fig_dir_metrics, exist_ok=True)

### Metrics on models

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

matplotlib.rcParams["pdf.fonttype"] = 42
matplotlib.rcParams["ps.fonttype"] = 42


SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

sns.color_palette("husl", 8)

hues = ["depth", "resolution"]

# select which results we want to plot
plot_eval_at_res = True
if plot_eval_at_res:
    df_plot = df_results_res
else:
    df_plot = df_results

# make plots
for hue in hues:
    for metric in ["precision", "recall", "accuracy"]:
        fig, ax = plt.subplots(1, 1, figsize=(6,4))
        sns.barplot(
            data=df_plot,
            x="width",
            y="precision",
            hue=hue,
            linestyle="-",
            linewidth=3,
            ax=ax,
        )
        plt.xlabel("Model Width")
        plt.ylabel("Precision")
        plt.ylim([0, 1.0])
        plt.title(f"Segmentation {metric.capitalize()} on Test Set")
        plt.grid()
        plt.tight_layout()
        plt.savefig(os.path.join(fig_dir_metrics, f"{metric}_by_width_and_{hue}.png"))
        plt.savefig(os.path.join(fig_dir_metrics, f"{metric}_by_width_and_{hue}.pdf"))
        plt.show()


### Examples from the different models

In [ ]:
# figure directory for saving
fig_dir_outputs = "figures/parametric/outputs"
os.makedirs(fig_dir_outputs, exist_ok=True)

In [ ]:
import matplotlib.colors as colors


# set colormaps
plt.set_cmap("Greys")
cmap_base = colors.LinearSegmentedColormap.from_list("wt", ["white", "teal"])
cmap_overlay = colors.LinearSegmentedColormap.from_list(
    "wtb", ["white", "teal", "darkslategrey"]
)

# get data
i_frame = 100

# set up the base figure file
fig_file_base = os.path.join(
    fig_dir_outputs,
    f"{{}}_frame_{i_frame}_{{}}",
)

# show the ground truth on the 512x512
print("Ground Truth Mask")
dataset_512 = datasets["width_16_depth_4_resolution_512"][1]
pc_img, gt_mask = dataset_512[i_frame]
gt_mask_np = gt_mask.detach().cpu().numpy()[0, ...]
plt.imshow(gt_mask_np, cmap=cmap_base)
plt.axis("off")
plt.savefig(fig_file_base.format("gt", "mask.png"))
plt.savefig(fig_file_base.format("gt", "mask.pdf"))
plt.show()

# show the image input
print("Image Input")
pc_img_np = pc_img.detach().cpu().numpy()[0, ...]
plt.imshow(pc_img_np > 0, cmap=cmap_overlay)
plt.axis("off")
plt.savefig(fig_file_base.format("input", "mask_img.png"))
plt.savefig(fig_file_base.format("input", "mask_img.pdf"))
plt.show()

# show model inferences
for model_name in models:
    print(model_name)
    ds = datasets[model_name][1]
    metadata = ds.get_metadata(i_frame)
    pc_img, gt_mask = ds[i_frame]
    pc_img = torch.unsqueeze(pc_img, 0)
    pc_np = ds.get_pointcloud(i_frame)

    # inference
    img_pred = models[model_name](pc_img, pc_np, metadata)

    # plotting
    threshold = 0.7
    img_pred_np = img_pred.detach().cpu().numpy()[0, 0, ...]
    plt.imshow(img_pred_np > threshold, cmap=cmap_base)
    plt.axis("off")
    plt.savefig(fig_file_base.format(model_name, "mask_pred.png"))
    plt.savefig(fig_file_base.format(model_name, "mask_pred.pdf"))
    plt.show()
